<a href="https://colab.research.google.com/github/limkaram/SKtelecom-TANGO-keyword-analysis/blob/master/TANGO_%EA%B5%AC%EC%B6%95_%EC%9E%91%EC%97%85_%ED%82%A4%EC%9B%8C%EB%93%9C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 목록 Mounting(매 코딩 시작전 해주어야함)
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('/content/drive/My Drive/Personal Project/TANGO 구축 작업 키워드 분석/20200101-20200731_TANGO 회선영향없음 키워드 주간 구축 작업.xlsx')
# 분석 예정 컬럼 = ['작업명']
data['작업명'].loc[0]

In [ ]:
# DATA 전처리


# 대괄호 내부 텍스트만 추출
import re
def get_regex_matching_text(text):
    text_in_square_backet_regex = re.compile(r'\[{1}.+\]{1}')
    result = text_in_square_backet_regex.findall(text)[0].replace('][', '|').replace('[', '').replace(']', '').split('|')
    return result

for name in data['작업명']:

test_text = 'T주-S0303 [5G-PON][장비_증설_회선영향없음]_SSKB구리_C12M1C53L(인장로주공)_H_인창한진AAUD_1_COT유니트 증설_건우통신(C200730002125)'
get_regex_matching_text(test_text)